In [25]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
import math

from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [26]:
# Read the data into a data frame
data = pd.read_csv('../input/telework-upper-predictions/data.csv')

In [27]:
new_data = data[['industry','occupation']]
X = new_data.values
y = data.estimated.values

In [28]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=0.2)

In [29]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(X_train, y_train, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
predictions = my_model.predict(X_test)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, y_test)))

Mean Absolute Error : 0.040156107880300905


In [64]:
import requests
def final_predictions(state=None,county=None,metropolitan=None):
    weightsind = [0.08, 0.19, 0.22, 0.52, 0.14, 0.19, 0.72, 0.76, 0.8, 0.3, 0.31, 0.41]
    weightsocc = [0.82,0.37,0.28,0.01,0.01]
    d2 = {}
    if state != None and county != None and metropolitan == None:
        url = "https://api.census.gov/data/2018/acs/acs5/profile?get=NAME,DP03_0001E,DP03_0033E,DP03_0034E,DP03_0035E,DP03_0036E,DP03_0037E,DP03_0038E,DP03_0039E,DP03_0040E,DP03_0041E,DP03_0042E,DP03_0043E,DP03_0044E&for=tract:*&in=state:"+str(state)+"%20county:"+str(county)+"&key=bf2a5abb0288a6187641bb539344cde1f7bdaf76"
        data = requests.get(url)
        output = data.json()[1:]
        for element in output:
            values = [int(x) for x in element[2:14]]
            if int(element[1]) != 0:
                  d2[element[0]] = []
                  d2[element[0]].append(np.dot(values, weightsind) / int(element[1]))
            else:
                  d2[element[0]] = []  
                  d2[element[0]].append(None)
        url = "https://api.census.gov/data/2018/acs/acs5/profile?get=NAME,DP03_0001E,DP03_0027E,DP03_0028E,DP03_0029E,DP03_0030E,DP03_0031E&for=tract:*&in=state:"+str(state)+"%20county:"+str(county)+"&key=bf2a5abb0288a6187641bb539344cde1f7bdaf76"

        data = requests.get(url)
        output = data.json()[1:]

        for element in output:
            values = [int(x) for x in element[2:7]]
            if int(element[1]) != 0:
                d2[element[0]].append(np.dot(values, weightsocc) / int(element[1]))
            else:
                d2[element[0]].append(None)

    return d2
   
   

d = final_predictions(state=42,county=101)

In [35]:
d.keys()

dict_keys(['Census Tract 359, Philadelphia County, Pennsylvania', 'Census Tract 362.01, Philadelphia County, Pennsylvania', 'Census Tract 362.02, Philadelphia County, Pennsylvania', 'Census Tract 106, Philadelphia County, Pennsylvania', 'Census Tract 244, Philadelphia County, Pennsylvania', 'Census Tract 246, Philadelphia County, Pennsylvania', 'Census Tract 249, Philadelphia County, Pennsylvania', 'Census Tract 258, Philadelphia County, Pennsylvania', 'Census Tract 265, Philadelphia County, Pennsylvania', 'Census Tract 267, Philadelphia County, Pennsylvania', 'Census Tract 268, Philadelphia County, Pennsylvania', 'Census Tract 319, Philadelphia County, Pennsylvania', 'Census Tract 330, Philadelphia County, Pennsylvania', 'Census Tract 340, Philadelphia County, Pennsylvania', 'Census Tract 346, Philadelphia County, Pennsylvania', 'Census Tract 348.01, Philadelphia County, Pennsylvania', 'Census Tract 61, Philadelphia County, Pennsylvania', 'Census Tract 84, Philadelphia County, Pennsyl

In [68]:
import copy
d
d2 = d
for key in d.keys():
    tempData = copy.copy(d[key])
    print(tempData)
    temp = np.array(tempData).reshape(1,-1)
    d2[key]=(my_model.predict(temp))

print(d2)

[0.1623763987792472, 0.20734689725330624]
[0.1926117936117936, 0.2685429975429975]
[0.2149414904537056, 0.2506282077602135]
[0.13195686900958464, 0.12552715654952076]
[0.16456002982848622, 0.1882289336316182]
[0.14469506292352372, 0.21311713455953538]
[0.14162812210915815, 0.16610083256244218]
[0.17820163487738422, 0.2386103542234332]
[0.15238767948124132, 0.19467114404817043]
[0.14848345764008414, 0.20001912411550968]
[0.18183513513513513, 0.24443513513513512]
[0.2150603808639108, 0.232954017649791]
[0.17125746399719005, 0.1882841587636108]
[0.19600086467790748, 0.2526242974492001]
[0.17921803127874886, 0.23851885924563015]
[0.18250935328701232, 0.20271512560128274]
[0.19908185361628408, 0.19504547423126894]
[0.1475878084179971, 0.21790711175616836]
[0.11650887573964498, 0.1755128205128205]
[0.16013698630136988, 0.16191780821917812]
[0.1691929133858268, 0.22451771653543304]
[0.36196958364497633, 0.5548342059336824]
[0.2730603267700042, 0.32917469627147045]
[0.10789727126805779, 0.1151

In [70]:
for key in d.keys():
    d[key] = d[key].item()

In [71]:
d

{'Census Tract 359, Philadelphia County, Pennsylvania': 0.2744278609752655,
 'Census Tract 362.01, Philadelphia County, Pennsylvania': 0.30814284086227417,
 'Census Tract 362.02, Philadelphia County, Pennsylvania': 0.34080469608306885,
 'Census Tract 106, Philadelphia County, Pennsylvania': 0.17800253629684448,
 'Census Tract 244, Philadelphia County, Pennsylvania': 0.21641674637794495,
 'Census Tract 246, Philadelphia County, Pennsylvania': 0.27800917625427246,
 'Census Tract 249, Philadelphia County, Pennsylvania': 0.2022114098072052,
 'Census Tract 258, Philadelphia County, Pennsylvania': 0.309766948223114,
 'Census Tract 265, Philadelphia County, Pennsylvania': 0.17636150121688843,
 'Census Tract 267, Philadelphia County, Pennsylvania': 0.23481085896492004,
 'Census Tract 268, Philadelphia County, Pennsylvania': 0.25675129890441895,
 'Census Tract 319, Philadelphia County, Pennsylvania': 0.3297674357891083,
 'Census Tract 330, Philadelphia County, Pennsylvania': 0.23916363716125488

In [75]:
import csv
with open('dict.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in d.items():
       writer.writerow([key, value])

In [ ]:
import json
json.dumps(d)